# Как работать с nc файлами?

## Чтение из файла:
**nc** - это формат файла для хранения многомерных метеорологических величин.

Помимо метеорологических данных в пс файле записана дополнительная информация:
- форма массива, которая может зависеть от широты, долготы, высоты и  времени.
- сами массивы значений широт, долгот, высот и  времени, которые удобно использовать при визуализации данных
- единицы измерения всех параметров

**netCDF4** -  специальный модуль для работы с файлами nc типа.

In [16]:
# из модуля импортируем Dataset
from netCDF4 import Dataset

# создаем объект класса Dataset, в качестве аргумента передаем имя файла, открываем в режиме (mode) чтения ('r')
# режим чтения можно не прописывать, т.к. он уже установлен по умолчанию.
nc = Dataset('Uzon_month_1958_2019_jra.nc', mode = 'r')

# теперь nc это объект/контейнер в котором содержится информация о многомерном массиве, 
# в частности о его размерности, название координатных осей, форма массива и т.д.

# получить доступ к информации, что содержится в nc можно с помощью variables
print(nc.variables)

{'lon': <class 'netCDF4._netCDF4.Variable'>
float64 lon(lon)
    standard_name: longitude
    long_name: longitude
    units: degrees_east
    axis: X
unlimited dimensions: 
current shape = (1,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lat': <class 'netCDF4._netCDF4.Variable'>
float64 lat(lat)
    standard_name: latitude
    long_name: latitude
    units: degrees_north
    axis: Y
unlimited dimensions: 
current shape = (36,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'lev': <class 'netCDF4._netCDF4.Variable'>
float64 lev(lev)
    long_name: generic
    units: level
    axis: Z
unlimited dimensions: 
current shape = (17,)
filling on, default _FillValue of 9.969209968386869e+36 used, 'time': <class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    standard_name: time
    units: hours since 1958-1-1 00:00:00
    calendar: standard
    axis: T
unlimited dimensions: time
current shape = (744,)
filling on, default _FillValue of 9.969209968386869e+36

## Что записано в содержание файла?
**название переменной** -->  'lat': <class 'netCDF4._netCDF4.Variable'>

    float64 lat(lat)
    standard_name: latitude   
    long_name: latitude
    units: degrees_north
    axis: Y
    unlimited dimensions: 
    current shape = (36,) <--- **форма**

Обращаем внимание на имя переменной, которое записано в кавычках -'lat' и ее длину/форму - 36 точек по оси широт. Имя переменной необходимо для создания массива широтных координат(т.е на каких широтах представлены данные)  

In [17]:
#импортируем numpy
import numpy as np

#  создаем переменную lat
lat = np.array(nc['lat'])
print(lat)

[-87.5 -82.5 -77.5 -72.5 -67.5 -62.5 -57.5 -52.5 -47.5 -42.5 -37.5 -32.5
 -27.5 -22.5 -17.5 -12.5  -7.5  -2.5   2.5   7.5  12.5  17.5  22.5  27.5
  32.5  37.5  42.5  47.5  52.5  57.5  62.5  67.5  72.5  77.5  82.5  87.5]


In [18]:
# аналогично для 'lev' и 'time' 
# высотные уровни
lev = np.array(nc['lev']),

# временная шкала
time = np.array(nc['time'])


    'uzon': <class 'netCDF4._netCDF4.Variable'>
    float32 uzon(time, lev, lat, lon)
    long_name: variables
    _FillValue: 2e+20
    missing_value: 2e+20
    unlimited dimensions: time
    current shape = (744, 17, 36, 1)
    filling on}

**'uzon' - название переменной, и ее форма (744, 17, 36, 1), в такой очередности записаны размерности (time, lev, lat, lon)**


In [19]:
# создаем переменную data, в которую записываем массив значений зональной скорости ветра.
data = np.array(nc['uzon'])

# закрываем файл
nc.close()


## Запись в файл:

Кроме значений метеовеличины nc файл должен быть дополнен кратким содержанием:
- что записано в файл,  в какой форме, название переменных, единицы измерения и т.д.

### Первый шаг -  нужно заполнить данную важную информацию созданием аттрибутов, размерностей и переменных!!!

In [20]:
from netCDF4 import Dataset
import numpy as np

# создаем объект(контейнер) с помощью Dataset в который нужно передать три параметра (аргумента) - 
# имя файла, режим и формат. 'w' - в данном режиме, если существует файл с похожим именем, 
# то хранящиеся данные будут стерты и запишутся новые данные.

ncfile = Dataset('new.nc', mode = 'w', format = 'NETCDF4_CLASSIC')

#### создание аттрибутов(аттрибуты - дополнительная информация, записывается по желанию создателя файла)

ncfile.title = 'My data'
ncfile.subtitle = 'Monthly mean'
ncfile.author = 'RSHU'
print(ncfile)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    title: My data
    subtitle: Monthly mean
    author: RSHU
    dimensions(sizes): 
    variables(dimensions): 
    groups: 


In [21]:
#### создание осей - dimensions(sizes)
# укажем форму наших осей массива с помощью метода createDimension
# У каждой оси есть имя(key) и длина(value). Обычно имя переменной это короткое обозначение. 
# эта информация записывается далее в словарь ncfile.dimensions

lat_dim = ncfile.createDimension('lat', 73)   
lon_dim = ncfile.createDimension('lon', 144)    
time_dim = ncfile.createDimension('time', 100) 

print(len(lat_dim))

# можно получить доступ к ключам и значениям словаря через
# ncfile.dimensions.items() - к ключам и значениям
# ncfile.dimensions.keys() - к ключам
# ncfile.dimensions.value() - к значениям

# for dim in ncfile.dimensions.items():
#     print(dim)

73


In [22]:
#### создание переменных - variables(dimensions):
# каждой переменной соответсвует ее имя, тип данных, форма (форма определеяется как кортеж из названия осей)
# переменные имеют аттрибуты, такие units(величина измерения), long_name (полное имя переменной)

# метод createVariables принимает три обязательных аргумента:
# - 1ый - название переменной (строка)
# - 2ой - тип данных
# - 3ий - кортеж, с названием оси (который должен быть уже создан)


lat = ncfile.createVariable('lat', np.float32, ('lat',))
lat.units = 'degrees_north'
lat.long_name = 'latitude'

lon = ncfile.createVariable('lon', np.float32, ('lon',))
lon.units = 'degrees_east'
lon.long_name = 'longitude'

time = ncfile.createVariable('time', np.float64, ('time',))
time.units = 'hours since 1800-01-01'
time.long_name = 'time'

# Объявляем 3х мерную переменную в которой будут записаны данные о температуре
temp = ncfile.createVariable('temp',np.float64,('lat','lon', 'time')) 
temp.units = 'K' 
temp.standard_name = 'air_temperature' 


### Второй шаг - поместим данные в переменные lat, lon, time, temp!

In [23]:
 
# определяем длину осей
nlats = len(lat_dim)
nlons = len(lon_dim)
ntimes = len(time_dim)

# заполняем переменную lat
lat[:] = -90. + (180./nlats)*np.arange(nlats) # от южного к северному
print(lat[:])

# 73 значений широт, расстояние между соседними широтами 2.46575

[-90.        -87.53425   -85.0685    -82.60274   -80.136986  -77.671234
 -75.20548   -72.73972   -70.27397   -67.80822   -65.34247   -62.876713
 -60.410957  -57.945206  -55.47945   -53.0137    -50.547947  -48.08219
 -45.61644   -43.150684  -40.684933  -38.219177  -35.753426  -33.28767
 -30.821918  -28.356165  -25.890411  -23.424658  -20.958904  -18.49315
 -16.027397  -13.561644  -11.09589    -8.630137   -6.1643834  -3.69863
  -1.2328767   1.2328767   3.69863     6.1643834   8.630137   11.09589
  13.561644   16.027397   18.49315    20.958904   23.424658   25.890411
  28.356165   30.821918   33.28767    35.753426   38.219177   40.684933
  43.150684   45.61644    48.08219    50.547947   53.0137     55.47945
  57.945206   60.410957   62.876713   65.34247    67.80822    70.27397
  72.73972    75.20548    77.671234   80.136986   82.60274    85.0685
  87.53425  ]


In [24]:
# заполняем переменные lon и time
lon[:] = (180./nlons)*np.arange(nlons) # Greenwich meridian eastward
time[:] = np.arange(ntimes)

# создадим 3хмерный массив случайных чисел
data_arr = np.random.uniform(low=280,high=330,size=(nlats,nlons,ntimes))

# запишем данные в переменную temp
temp[:,:,:] = data_arr 

# закрываем файл
ncfile.close()
print('Dataset is closed!')

# Поздравляю вы записали данные в файл!

Dataset is closed!
